<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания №19


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс Subscription в C#, который будет представлять подписки на
различные услуги. На основе этого класса разработать 2-3 производных класса,
демонстрирующих принципы наследования и полиморфизма. В каждом из классов
должны быть реализованы новые атрибуты и методы, а также переопределены
некоторые методы базового класса для демонстрации полиморфизма.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) исользуйтие в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [2]:
using System;
using System.Collections.Generic;
using System.Linq; 

// --- Шаг 2: Определение делегатов и событий ---

// определяем делегат, который будет служить подписью для обработчиков событий
// он будет передавать информацию о том, какая подписка изменила статус и какой новый статус
public delegate void StatusChangedHandler(Subscription sender, bool newStatus);


// интерфейс для любого объекта, который можно логировать
public interface ILoggable
{
    void LogStatus();
}

// основной интерфейс для всех подписок, будет использоваться для управления зависимостями
public interface ISubscription
{
    string ServiceName { get; }
    double Cost { get; }
    string GetSubscriptionDetails();
}


// интерфейс для подписок, к которым можно применить промокод
public interface IPromoCodeApplicable
{
    // метод для применения промокода
    void ApplyPromoCode(string promoCode);
}

// интерфейс для сервисов, имеющих профиль пользователя
public interface IUserProfile
{
    // свойство для имени пользователя
    string UserName { get; set; }
    // метод для обновления профиля
    void UpdateProfile(string newName);
}


// базовый класс для подписок, теперь реализует ISubscription
public class Subscription : ISubscription
{
    public int SubscriptionId { get; set; }
    public string ServiceName { get; set; }
    public double Cost { get; set; }
    
    public DateTime StartDate { get; private set; } // дата начала подписки
    public bool IsActive { get; private set; }      // статус активности
    
    public string BillingCycle { get; set; } // цикл оплаты (ежемесячно, ежегодно)
    
    // объявляем событие, на которое смогут подписываться другие части программы
    public event StatusChangedHandler StatusChanged;

    public Subscription(int id, string serviceName, double cost)
    {
        SubscriptionId = id;
        ServiceName = serviceName;
        Cost = cost;
        StartDate = DateTime.Now; 
        IsActive = true;
        BillingCycle = "Ежемесячно"; // значение по умолчанию
    }

    public void ApplyDiscountFrom(Subscription otherSubscription, double discountPercentage)
    {
        double discountAmount = otherSubscription.Cost * (discountPercentage / 100);
        this.Cost -= discountAmount;
        Console.WriteLine($"Применена скидка {discountPercentage}% от подписки '{otherSubscription.ServiceName}'. Новая стоимость '{this.ServiceName}': {this.Cost:C}");
    }
    
    public void CancelSubscription()
    {
        IsActive = false;
        Console.WriteLine($"Подписка на '{ServiceName}' была отменена");
        // вызываем событие, чтобы уведомить всех подписчиков об изменении статуса
        StatusChanged?.Invoke(this, false); // false означает "неактивна"
    }
    
    public void Reactivate()
    {
        IsActive = true;
        Console.WriteLine($"Подписка на '{ServiceName}' снова активна.");
        // также уведомляем подписчиков о возобновлении
        StatusChanged?.Invoke(this, true); // true означает "активна"
    }
    
    public int GetSubscriptionAgeInDays()
    {
        return (int)(DateTime.Now - StartDate).TotalDays;
    }

    public virtual double CalculateMonthlyCost()
    {
        return Cost;
    }

    public virtual void ExtendSubscription(int months)
    {
        Console.WriteLine($"Подписка на '{ServiceName}' продлена на {months} мес.");
    }
    
    public void ExtendSubscription()
    {
        ExtendSubscription(1); 
    }
    
    public void ExtendSubscription(int months, double bonusDiscount)
    {
        Console.WriteLine($"Подписка на '{ServiceName}' продлена на {months} мес. с дополнительной скидкой {bonusDiscount}%!");
    }


    public virtual string GetSubscriptionDetails()
    {
        return $"ID: {SubscriptionId}, Услуга: {ServiceName}, Стоимость: {Cost:C} ({BillingCycle}), Активна: {IsActive}";
    }
}

// класс реализует несколько интерфейсов, включая ILoggable
public class OnlineServiceSubscription : Subscription, IUserProfile, IPromoCodeApplicable, ILoggable
{
    public int MaxUsers { get; set; }
    public double UsedStorageGB { get; set; } 
    public string UserName { get; set; } 
    
    public string ServiceRegion { get; set; } // регион серверов

    public OnlineServiceSubscription(int id, string serviceName, double costPerUser, int maxUsers, string userName)
        : base(id, serviceName, costPerUser)
    {
        MaxUsers = maxUsers;
        UserName = userName;
        UsedStorageGB = 0; 
        ServiceRegion = "EU-West"; // значение по умолчанию
    }
    
    public void CheckStorageUsage()
    {
        Console.WriteLine($"Использование хранилища: {UsedStorageGB} GB");
    }
    
    public void ChangeRegion(string newRegion)
    {
        ServiceRegion = newRegion;
        Console.WriteLine($"Регион сервиса изменен на {ServiceRegion}");
    }

    public void UpdateProfile(string newName)
    {
        Console.WriteLine($"Имя пользователя для '{ServiceName}' изменено с '{UserName}' на '{newName}'");
        UserName = newName;
    }

    public void ApplyPromoCode(string promoCode)
    {
        if (promoCode == "CLOUD20")
        {
            Cost *= 0.8; 
            Console.WriteLine($"Промокод 'CLOUD20' применен! Новая базовая стоимость: {Cost:C}");
        }
        else
        {
            Console.WriteLine("Неверный промокод");
        }
    }

    public override double CalculateMonthlyCost()
    {
        return Cost * MaxUsers;
    }
    
    void ILoggable.LogStatus()
    {
        Console.WriteLine($"[ЛОГ]: Статус сервиса '{ServiceName}': Активен, Пользователь: {UserName}, Хранилище: {UsedStorageGB} GB.");
    }
}

public class StreamingSubscription : Subscription, IPromoCodeApplicable
{
    public int MaxStreams { get; set; }
    public bool HasOfflineMode { get; set; } 

    public StreamingSubscription(int id, string serviceName, double cost, int maxStreams, bool hasOfflineMode)
        : base(id, serviceName, cost)
    {
        MaxStreams = maxStreams;
        HasOfflineMode = hasOfflineMode;
    }
    
    public void ToggleOfflineMode()
    {
        HasOfflineMode = !HasOfflineMode;
        Console.WriteLine($"Офлайн-режим для '{ServiceName}' теперь {(HasOfflineMode ? "включен" : "выключен")}");
    }
    
    public void ApplyPromoCode(string promoCode)
    {
        if (promoCode == "MUSIC15")
        {
            Cost *= 0.85; 
            Console.WriteLine($"Промокод 'MUSIC15' применен! Новая базовая стоимость: {Cost:C}");
        }
        else
        {
            Console.WriteLine("Неверный промокод");
        }
    }

    public override void ExtendSubscription(int months)
    {
        Console.WriteLine($"Подписка на '{ServiceName}' продлена на {months} мес. Специальное предложение: +1 месяц в подарок!");
    }
}

public class VideoSubscription : Subscription
{
    public string VideoQuality { get; set; }
    public List<string> IncludedChannels { get; set; }

    public VideoSubscription(int id, string serviceName, double cost, string videoQuality)
        : base(id, serviceName, cost)
    {
        VideoQuality = videoQuality;
        IncludedChannels = new List<string>(); 
    }
    
    public void DisplayChannels()
    {
        Console.WriteLine("Включенные каналы:");
        foreach (var channel in IncludedChannels)
        {
            Console.WriteLine($"- {channel}");
        }
    }

    public override string GetSubscriptionDetails()
    {
        return base.GetSubscriptionDetails() + $", Качество видео: {VideoQuality}";
    }
}

public class PremiumVideoSubscription : VideoSubscription
{
    public bool HasPersonalManager { get; set; } 

    public PremiumVideoSubscription(int id, string serviceName, double cost, string videoQuality, bool hasManager)
        : base(id, serviceName, cost, videoQuality) 
    {
        HasPersonalManager = hasManager;
    }
    
    public void ContactManager()
    {
        if (HasPersonalManager)
        {
            Console.WriteLine("Соединение с вашим персональным менеджером...");
        }
        else
        {
            Console.WriteLine("Эта услуга не включена в вашу подписку");
        }
    }

    public override string GetSubscriptionDetails()
    {
        return base.GetSubscriptionDetails() + $", Персональный менеджер: {(HasPersonalManager ? "Да" : "Нет")}";
    }
}

// ----- ОСНОВНАЯ ПРОГРАММА -----

// --- Шаг 1: Использование коллекций ---

// используем List<T> для хранения всех подписок (полиморфизм)
List<Subscription> subscriptions = new List<Subscription>();

var onlineStorage = new OnlineServiceSubscription(101, "Облачное хранилище", 5.0, 10, "User123");
var musicService = new StreamingSubscription(202, "Музыкальный сервис", 15.0, 4, true);
var premiumVideo = new PremiumVideoSubscription(404, "Премиум Кино", 40.0, "8K HDR", true);
premiumVideo.IncludedChannels.Add("Новости Кино");
premiumVideo.IncludedChannels.Add("Классика");

subscriptions.Add(onlineStorage);
subscriptions.Add(musicService);
subscriptions.Add(premiumVideo);

// используем Dictionary<TKey, TValue> для быстрого доступа к подпискам по их ID
Dictionary<int, Subscription> subscriptionRegistry = new Dictionary<int, Subscription>();
foreach(var sub in subscriptions)
{
    subscriptionRegistry.Add(sub.SubscriptionId, sub);
}

// метод-обработчик для события изменения статуса
static void HandleStatusChange(Subscription sender, bool newStatus)
{
    Console.WriteLine($"[СИСТЕМА УВЕДОМЛЕНИЙ]: Внимание! Статус подписки '{sender.ServiceName}' изменился. Новый статус: {(newStatus ? "АКТИВНА" : "НЕАКТИВНА")}.");
}


// ----- Демонстрация работы -----

// --- демонстрация делегатов и лямбда-выражений ---
Console.WriteLine("--- Демонстрация Делегатов (через LINQ с лямбда-выражениями) ---\n");
// используем делегат Func<T, bool> в методе Where для фильтрации списка
var expensiveSubscriptions = subscriptions.Where(s => s.Cost > 20.0);

Console.WriteLine("Найдены дорогие подписки (стоимость > 20):");
foreach(var sub in expensiveSubscriptions)
{
    Console.WriteLine(sub.GetSubscriptionDetails());
}

// --- демонстрация событий ---
Console.WriteLine("\n\n--- Демонстрация Событий ---\n");

// подписываем метод-обработчик на событие у подписки на музыку
musicService.StatusChanged += HandleStatusChange;

// подписываемся на событие у другой подписки, но с помощью лямбда-выражения
onlineStorage.StatusChanged += (sender, status) => 
{
    Console.WriteLine($"[Email]: Уважаемый {((OnlineServiceSubscription)sender).UserName}, ваша подписка '{sender.ServiceName}' теперь {(status ? "активна" : "неактивна")}.");
};


Console.WriteLine("Отменяем подписку на музыку (это вызовет событие)...");
musicService.CancelSubscription();

Console.WriteLine("\nОтменяем подписку на хранилище (это вызовет другое событие)...");
onlineStorage.CancelSubscription();

Console.WriteLine("\nВозобновляем подписку на хранилище...");
onlineStorage.Reactivate();

// --- демонстрация коллекций (поиск по словарю) ---
Console.WriteLine("\n\n--- Демонстрация Коллекций (поиск в Dictionary) ---\n");
int idToFind = 404;
if (subscriptionRegistry.TryGetValue(idToFind, out Subscription foundSub))
{
    Console.WriteLine($"Поиск по ID {idToFind}... Найдена подписка:");
    Console.WriteLine(foundSub.GetSubscriptionDetails());
}
else
{
    Console.WriteLine($"Подписка с ID {idToFind} не найдена.");
}

--- Демонстрация Делегатов (через LINQ с лямбда-выражениями) ---

Найдены дорогие подписки (стоимость > 20):
ID: 404, Услуга: Премиум Кино, Стоимость: ¤40.00 (Ежемесячно), Активна: True, Качество видео: 8K HDR, Персональный менеджер: Да


--- Демонстрация Событий ---

Отменяем подписку на музыку (это вызовет событие)...
Подписка на 'Музыкальный сервис' была отменена
[СИСТЕМА УВЕДОМЛЕНИЙ]: Внимание! Статус подписки 'Музыкальный сервис' изменился. Новый статус: НЕАКТИВНА.

Отменяем подписку на хранилище (это вызовет другое событие)...
Подписка на 'Облачное хранилище' была отменена
[Email]: Уважаемый User123, ваша подписка 'Облачное хранилище' теперь неактивна.

Возобновляем подписку на хранилище...
Подписка на 'Облачное хранилище' снова активна.
[Email]: Уважаемый User123, ваша подписка 'Облачное хранилище' теперь активна.


--- Демонстрация Коллекций (поиск в Dictionary) ---

Поиск по ID 404... Найдена подписка:
ID: 404, Услуга: Премиум Кино, Стоимость: ¤40.00 (Ежемесячно), Активна: Tru